---
title: 'Custas'
author: 'Philipp Kleer'
format: dashboard
theme: [sandstone, ./assets/dashboard.scss]
server: shiny
execute: 
  eval: true
  echo: false
  kernel: python3
---

In [ ]:
#| label: preparations
import plotly.express as px
import pandas as pd
from shiny import reactive, render
from shiny.express import input, render, ui
from shinywidgets import render_widget
from datetime import datetime, timedelta
from faicons import icon_svg

df = pd.read_csv('data/custas.csv')
# df['data'] = pd.to_datetime(df['data'])
# df['data'] = df['data'].dt.strftime('%Y-%m')

grouped_df = df.loc[(df['tipo'] != 'renda')].groupby(
    ['data', 'tipo']).agg({'valor': 'sum'}).reset_index()

df_avg = grouped_df.groupby('tipo').agg({'valor': 'mean'}).reset_index()

# Overview

## Row {height=40%}

### {.sidebar}


In [ ]:
#| label: input-select-overview
newest_month = df['data'].max()
oldest_month = df['data'].min()

newest_month_str = pd.to_datetime(newest_month).strftime('%Y-%m-%d')
oldest_month_str = pd.to_datetime(oldest_month).strftime('%Y-%m-%d')

ui.input_date_range(
    "daterange", "Date range:",
    start=oldest_month_str,
    end=newest_month_str,
    min=oldest_month_str,
    max=newest_month_str,
    format='yyyy-mm',
    startview = "year",
    language='pt-BR'
)

@reactive.calc
def filtered_data():
    data_selected = df.loc[(df['data'] >= input.daterange()[0].strftime('%Y-%m')) & (df['data'] <= input.daterange()[1].strftime('%Y-%m'))]
    return data_selected

### Column


In [ ]:
@render.text  
def medio_1():  
    data_selected = df.loc[(df['data'] >= input.daterange()[0].strftime('%Y-%m')) & (df['data'] <= input.daterange()[1].strftime('%Y-%m')) & (df['tipo'] == 'apartamento')]
    mean_value = data_selected.groupby('data')['valor'].sum()

    return f"R$ {mean_value.mean():.2f}"  

### Column

In [ ]:
@render.text  
def medio_2():  
    data_selected = df.loc[(df['data'] >= input.daterange()[0].strftime('%Y-%m')) & (df['data'] <= input.daterange()[1].strftime('%Y-%m')) & (df['tipo'] == 'supermercado')]
    mean_value = data_selected.groupby('data')['valor'].sum()

    return f"R$ {mean_value.mean():.2f}" 

### Column

In [ ]:
@render.text  
def medio_3():  
    data_selected = df.loc[(df['data'] >= input.daterange()[0].strftime('%Y-%m')) & (df['data'] <= input.daterange()[1].strftime('%Y-%m')) & (df['tipo'] == 'lazer')]
    mean_value = data_selected.groupby('data')['valor'].sum()

    return f"R$ {mean_value.mean():.2f}" 

## Row {height=60%}

### Column {width=45%}

In [ ]:
#| label: lineplot
@render_widget
def plot_line():  
    plotdata = filtered_data()
    
    plotdata_cleaned = plotdata.loc[(plotdata['tipo'] != 'renda')].groupby(
    ['data', 'tipo']).agg({'valor': 'sum'}).reset_index()


    line_plot = px.line(
        data_frame=plotdata_cleaned.loc[(df['tipo'] != 'renda')], 
        x="data", 
        y='valor', 
        color='tipo',
        labels=dict(data="Mês", valor="Valor (R$)", tipo="Tipo"),
        color_discrete_map={
            'apartamento':'#D81B60',
            'lazer':'#1E88E5',
            'outros':'#FFC107',
            'supermercado':'#009E73'
        }
    ).update_layout(
        xaxis_type='category',
    )

    line_plot.update_xaxes(
        tickangle=60,
        title_standoff=25
    )    

    line_plot.update_yaxes(
        tick0=0, dtick=250
    )

    return line_plot

### Column {width=45%}

In [ ]:
#| label: pieplot
@render_widget
def plot():  
    plotdata = filtered_data()

    pie_total = px.pie(
        plotdata.loc[(df['tipo'] != 'renda')],
        values='valor',  
        names='tipo', 
        hole=0.5,
        color='tipo', 
        color_discrete_map={
            'apartamento':'#D81B60',
            'lazer':'#1E88E5',
            'outros':'#FFC107',
            'supermercado':'#009E73'
        }
        )
    return pie_total

# Monthly

## Row 
### {.sidebar}


In [ ]:
#| label: input-select
def generate_month_range(start_month: str, end_month: str):
    # Convert the start and end months to datetime objects
    start_date = datetime.strptime(start_month, "%Y-%m")
    end_date = datetime.strptime(end_month, "%Y-%m")

    # Initialize the array and a temporary variable to track the current month
    month_array = []
    current_date = start_date

    # Loop through each month from start to end, appending to the list
    while current_date <= end_date:
        month_array.append(current_date.strftime("%Y-%m"))  # Append as string
        # Move to the next month
        current_date = (current_date.replace(day=1) + timedelta(days=31)).replace(day=1)

    return month_array

@reactive.effect
def _():
    start_month = input.daterange()[0].strftime('%Y-%m')
    end_month = input.daterange()[1].strftime('%Y-%m')

    # Generate the month range
    months = generate_month_range(start_month, end_month)

    input.month_select.choices = months

    ui.insert_ui

    ui.update_select(
        "month",
        choices=input.month_select.choices,
        selected=input.month_select.choices[1]
    )

ui.input_select(
    "month", "Select month", []
)

From the time frame selected in `overview` you can select a month to make the overviews for that specific month.

### Column

In [ ]:
#| label: pieplot_month
@render_widget
def plot_month():  
    pie_total = px.pie(
        data_frame=df.loc[(df['data'] == str(input.month())) & (df['tipo'] != 'renda')],
        values='valor',  
        names='tipo', 
        color='tipo', 
        color_discrete_map={
            'apartamento':'#D81B60',
            'lazer':'#1E88E5',
            'outros':'#FFC107',
            'supermercado':'#009E73'
        },
        hole=0.5)
    return pie_total

## Row {height=10%}

### Column

In [ ]:
@render.text
def valor_apartamento():
    data = df.loc[(df['data'] == str(input.month())) & (df['tipo'].isin(['apartamento']))]

    sum_value = round(data.valor.sum(), 2)
    
    return f'Gastos para apartamento: {sum_value}'

### Column


In [ ]:
@render.text
def valor_supermercado():
    data = df.loc[(df['data'] == str(input.month())) & (df['tipo'].isin(['supermercado']))]

    sum_value = round(data.valor.sum(), 2)
    
    return f'Gastos para supermercado: {sum_value}'

### Column

In [ ]:
@render.text
def valor_lazer():
    data = df.loc[(df['data'] == str(input.month())) & (df['tipo'].isin(['lazer']))]

    sum_value = round(data.valor.sum(), 2)
    
    return f'Gastos para lazer: {sum_value}'

### Column

In [ ]:
@render.text
def valor_outros():
    data = df.loc[(df['data'] == str(input.month())) & (df['tipo'].isin(['outros']))]

    sum_value = round(data.valor.sum(), 2)
    
    return f'Gastos para outros: {sum_value}'

## Row {height=40%}

### Column


In [ ]:
#| label: pieplot_apartamento
@render_widget
def plot_apartamento():  
    pie_apartamento = px.pie(
        data_frame=df.loc[(df['data'] == str(input.month())) & (df['tipo'].isin(['apartamento']))], 
        values='valor',  
        names='label', 
        hole=0.5,
        color='label', 
        color_discrete_map={
            'Aluguél':'#D81B60',
            'Energia':'#D81B6090',
            'Oi! Fibra':'#D81B6060',
            'Água':'#D81B6030',
            'Gás': '#D81B6010'
        }
        )
    return pie_apartamento

### Column


In [ ]:
#| label: pieplot_supermercado
@render_widget
def plot_supermercado():  
    pie_supermercado = px.pie(
        data_frame=df.loc[(df['data'] == str(input.month())) & (df['tipo'] == 'supermercado')], 
        values='valor',  
        names='label', 
        color='label', 
        color_discrete_map={
            'Arco-Mix':'#009E7390',
            'Atacado':'#009E7360',
            'Farmácia':'#009E7330'
        },
        hole=0.5)
    return pie_supermercado

### Column


In [ ]:
#| label: pieplot_lazer
@render_widget
def plot_lazer():  
    pie_lazer = px.pie(
        data_frame=df.loc[(df['data'] == str(input.month())) & (df['tipo'] == 'lazer')], 
        values='valor',  
        names='label', 
        color='label', 
        color_discrete_map={
            'Bebidas/Comidas':'#1E88E590',
            'iFood':'#1E88E560',
            'Museu/Cultura/Cinema':'#1E88E530'
        },
        hole=0.5)
    return pie_lazer

### Column


In [ ]:
#| label: pieplot_outros
@render_widget
def plot_outros():  
    pie_outros = px.pie(
        data_frame=df.loc[(df['data'] == str(input.month())) & (df['tipo'] == 'outros')], 
        values='valor',  
        names='label', 
        color='label', 
        color_discrete_map={
            'Uber':'#1E88E590',
            'Assinaturas':'#1E88E560',
            'Mobiliário': '#1E88E530',
            'Parcelamento Pai':'#1E88E505'
        },
        hole=0.5)
    return pie_outros

# Data

In [ ]:
#| label: data-exhibition

ui.h2("Data from the selected months")

@render.data_frame  
def penguins_df():
    return render.DataGrid(filtered_data()) 